# Simple Sentinel-2 NDWI Analysis

This notebook demonstrates how to download Sentinel-2 satellite data and calculate the Normalized Difference Water Index (NDWI) using the STAC API.

## What is NDWI?

**NDWI (Normalized Difference Water Index)** is calculated as:
```
NDWI = (Green - NIR) / (Green + NIR)
```

Where:
- **Green** = Green band (Band 3 in Sentinel-2)
- **NIR** = Near-Infrared band (Band 8 in Sentinel-2)

NDWI values typically range from -1 to 1, with:
- **Positive values** indicating water bodies
- **Negative values** indicating non-water features

## No Authentication Required!

This approach uses the public STAC API hosted on AWS - no accounts or credentials needed!


## Step 1: Import Libraries and Setup


In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Add parent directory to path to import our modules
sys.path.append('..')

from stac_downloader import STACDownloader
from simple_config import get_location, get_date_range, print_available_locations

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


## Step 2: Choose Your Location

You can either use a predefined location or enter custom coordinates.


In [ ]:
# Show available predefined locations
print("📍 Available predefined locations:")
print_available_locations()
print()

# Choose a location (modify this as needed)
location_key = "new_york"  # Change this to any key from the list above

# Get location details
location = get_location(location_key)
lon, lat = location['lon'], location['lat']

print(f"✅ Selected: {location['name']}")
print(f"   Coordinates: {lat:.4f}°N, {lon:.4f}°E")
print(f"   Description: {location['description']}")


### Alternative: Use Custom Coordinates

If you want to analyze a different location, uncomment and modify the code below:


In [ ]:
# Uncomment and modify these lines for custom coordinates
# lat = 40.78  # Your latitude
# lon = -73.97  # Your longitude
# location_name = "My Custom Location"
# 
# print(f"📍 Custom location: {lat:.4f}°N, {lon:.4f}°E")


## Step 3: Download and Analyze Sentinel-2 Data

Now let's download the Sentinel-2 data and calculate NDWI for your chosen location.


In [ ]:
# Initialize the STAC downloader
downloader = STACDownloader()

# Get date range (last 30 days)
start_date, end_date = get_date_range(days_back=30)

print(f"🔍 Analysis Parameters:")
print(f"   Location: {lat:.4f}°N, {lon:.4f}°E")
print(f"   Date Range: {start_date} to {end_date}")
print(f"   Search radius: 2.0 km")
print(f"   Max cloud cover: 20%")
print()

# Download and calculate NDWI
print("🚀 Starting download and analysis...")
results = downloader.download_and_calculate_ndwi(
    lon=lon,
    lat=lat,
    start_date=start_date,
    end_date=end_date,
    buffer_km=2.0,  # 2km radius around the point
    cloud_cover_max=20
)

print("\n✅ Download and analysis completed!")
print(f"📁 Files created:")
for key, filepath in results.items():
    if key != 'item':
        print(f"   {key}: {filepath}")


## Step 4: Analyze Results

Let's examine the NDWI results and understand what they tell us about water presence.


In [ ]:
# Analyze NDWI statistics
stats = downloader.analyze_ndwi(results['ndwi'])

print("📊 NDWI Analysis Results:")
print(f"   Mean NDWI: {stats['mean']:.3f}")
print(f"   Min NDWI: {stats['min']:.3f}")
print(f"   Max NDWI: {stats['max']:.3f}")
print(f"   Standard deviation: {stats['std']:.3f}")
print(f"   Water percentage: {stats['water_percentage']:.1f}%")
print(f"   Water pixels: {stats['water_pixels']:,}")
print(f"   Total pixels: {stats['total_pixels']:,}")

# Interpretation
print("\n💡 Interpretation:")
if stats['water_percentage'] > 50:
    print("   🌊 High water content detected - likely a water body")
elif stats['water_percentage'] > 20:
    print("   💧 Moderate water content - mixed water/land area")
elif stats['water_percentage'] > 5:
    print("   🌿 Low water content - mostly land with some water features")
else:
    print("   🏔️  Very low water content - mostly land/urban area")


## Step 5: Visualize Results

Let's create visualizations to see the NDWI results and compare them with the RGB image.


In [ ]:
# Create visualization
print("📊 Creating visualization...")
fig = downloader.plot_results(results['ndwi'], results['rgb'])
plt.show()

print("\n📈 NDWI Color Scale:")
print("   🔴 Red: High NDWI values (likely water)")
print("   🟡 Yellow: Medium NDWI values (mixed areas)")
print("   🔵 Blue: Low NDWI values (likely land)")
